# Analyze the annotation results

### Please adjust `data_dir` to your path

In [1]:
import os
import pandas as pd
from data_analysis.core.dataset import AI4ODDataset
from data_analysis.core.filter import NumAnnotatorFilter

data_dir = ""
assert os.path.isdir(data_dir), "Your data directory is not a directory."

filters = [NumAnnotatorFilter(2)]
dataset = AI4ODDataset(data_dir, load_images=False, filters=filters)

# get list of all annotators
annotator_ids = []
for img_info in dataset.img_infos:
    ids = list(img_info.annots.keys())
    annotator_ids += ids

annotator_ids = sorted(list(set(annotator_ids)))
print("Annotator IDs:", annotator_ids)

print("Number of images in dataset:", len(dataset))

num_annotators_per_scene = [0] * 4
for scene in dataset.scenes:
    num_annots = len(scene.img_infos[0].annots)
    
    if num_annots == 1:
        print(scene)
    num_annotators_per_scene[num_annots - 1] += 1

print("Scenes with 1 annotator:\t", num_annotators_per_scene[0])
print("Scenes with 2 annotators:\t", num_annotators_per_scene[1])
print("Scenes with 3 annotators:\t", num_annotators_per_scene[2])
print("Scenes with 4 annotators:\t", num_annotators_per_scene[3])

Annotator IDs: [0, 1, 2, 3, 4, 5, 6, 8, 9, 10]
Number of images in dataset: 3911
Scenes with 1 annotator:	 0
Scenes with 2 annotators:	 70
Scenes with 3 annotators:	 23
Scenes with 4 annotators:	 0


In [3]:
img_info_df = pd.DataFrame(columns=["img_info", "num_annotators"])

for img_info in dataset.img_infos:
    num_annotators = len(img_info.annots)
    img_info_df.loc[len(img_info_df)] = [img_info, num_annotators]


In [4]:
num_directs = 0
num_indirects = 0
for img_info in dataset.img_infos:
    for annot in img_info.annots.values():
        num_indirects += len(annot.reflections)
        num_directs += len([v for v in annot.vehicles if v.direct])

print("Number of light reflection annotations:", num_indirects)
print("Number of direct vehicle annotations:", num_directs)

Number of light reflection annotations: 25109
Number of direct vehicle annotations: 3602


## Original Bounding Box Annotations

### Number of bounding boxes per image


In [5]:
from data_analysis.boxes.utils import count_num_boxes

img_ids = [img_info.img_name.split(".")[0] for img_info in dataset.img_infos]
count_box_df_indirect = pd.DataFrame(columns=annotator_ids, index=img_ids)
count_box_df_direct = pd.DataFrame(columns=annotator_ids, index=img_ids)

for img_info in dataset.img_infos:
    img_id = img_info.img_name.split(".")[0]
    for annotator_id, annot in img_info.annots.items():
        num_boxes_indirect = count_num_boxes(annot, indirect=True, direct=False)
        count_box_df_indirect.loc[img_id][annotator_id] = num_boxes_indirect

        num_boxes_direct = count_num_boxes(annot, indirect=False, direct=True)
        count_box_df_direct.loc[img_id][annotator_id] = num_boxes_direct

In [6]:
diff_direct = count_box_df_direct.max(axis="columns") - count_box_df_direct.min(axis="columns") 

diff_indirect = count_box_df_indirect.max(axis="columns") - count_box_df_indirect.min(axis="columns")

mean_diff_direct = diff_direct.mean()
mean_diff_indirect = diff_indirect.mean()

print("Mean difference in number of bounding boxes")
print("Direct:\t\t", round(mean_diff_direct, 2))
print("Indirect:\t", round(mean_diff_indirect, 2))

Mean difference in number of bounding boxes
Direct:		 0.06
Indirect:	 1.51


## Matching Boxes (non-exclusive) & IoUs

In [7]:
import statistics
from data_analysis.boxes.match import get_matches

direct_matches = {}
direct_unmatches = {}

indirect_matches = {}
indirect_unmatches = {}

# new columns
img_info_df["direct_match_exclusive"] = None
img_info_df["direct_unmatch_exclusive"] = None

img_info_df["direct_match_non_exclusive"] = None
img_info_df["direct_unmatch_non_exclusive"] = None

img_info_df["indirect_match_exclusive"] = None
img_info_df["indirect_unmatch_exclusive"] = None

img_info_df["indirect_match_non_exclusive"] = None
img_info_df["indirect_unmatch_non_exclusive"] = None


for i, row in img_info_df.iterrows():
    img_info = row["img_info"]
    num_annotators = row["num_annotators"]
    
    if num_annotators > 1:
        # non-exclusive indirect
        matches, unmatches = get_matches(img_info, direct=False, indirect=True, iou_thresh=0.01, exclusive=False)
        img_info_df.at[i, "indirect_match_non_exclusive"] = matches
        img_info_df.at[i, "indirect_unmatch_non_exclusive"] = unmatches

        # non-exclusive direct
        matches, unmatches = get_matches(img_info, direct=True, indirect=False, iou_thresh=0.01, exclusive=False)
        img_info_df.at[i, "direct_match_non_exclusive"] = matches
        img_info_df.at[i, "direct_unmatch_non_exclusive"] = unmatches

        # exclusive indirect
        matches, unmatches = get_matches(img_info, direct=False, indirect=True, iou_thresh=0.01, exclusive=True)
        img_info_df.at[i, "indirect_match_exclusive"] = matches
        img_info_df.at[i, "indirect_unmatch_exclusive"] = unmatches

        # exclusive direct
        matches, unmatches = get_matches(img_info, direct=True, indirect=False, iou_thresh=0.01, exclusive=True)
        img_info_df.at[i, "direct_match_exclusive"] = matches
        img_info_df.at[i, "direct_unmatch_exclusive"] = unmatches

        matches, unmatches = get_matches(img_info, direct=True, indirect=False, iou_thresh=0.01, exclusive=False)
        direct_matches[img_info.img_name] = matches
        direct_unmatches[img_info.img_name] = unmatches


In [8]:

indirect_ious = []
direct_ious = []

# indirect_unmatches_per_img = []
# indirect_matches_per_img = []

num_exclusive_indirect = []
num_exclusive_direct = []

num_matches_non_exclusive_direct = []
num_matches_non_exclusive_indirect = []

num_unmatches_non_exclusive_direct = []
num_unmatches_non_exclusive_indirect = []

indirect_match_overlapping = []
direct_match_overlapping = []

for i, row in img_info_df.iterrows():
    if row["indirect_match_non_exclusive"] or row["indirect_unmatch_non_exclusive"]:
        indirect_ious += [item[1] for item in row["indirect_match_non_exclusive"]]
        num_matches_non_exclusive_indirect.append(len(row["indirect_match_non_exclusive"]))
        num_unmatches_non_exclusive_indirect.append(len(row["indirect_unmatch_non_exclusive"]))
        
    if row["indirect_match_exclusive"]:
        num_exclusive_direct.append(len(row["indirect_match_exclusive"]))
    
    if row["direct_match_non_exclusive"] or row["direct_unmatch_non_exclusive"]:
        direct_ious += [item[1] for item in row["direct_match_non_exclusive"]]
        num_matches_non_exclusive_direct.append(len(row["direct_match_non_exclusive"]))
        num_unmatches_non_exclusive_direct.append(len(row["direct_unmatch_non_exclusive"]))
        
    if row["direct_match_exclusive"]:
        num_exclusive_direct.append(len(row["direct_match_exclusive"]))
    
    if row["indirect_match_non_exclusive"] or row["indirect_match_exclusive"]:
        exclusive = row["indirect_match_exclusive"]
        exclusive = exclusive if exclusive else []
        
        non_exclusive = row["indirect_match_non_exclusive"]
        non_exclusive = non_exclusive if non_exclusive else []
        
        indirect_match_overlapping.append(len(non_exclusive) - len(exclusive))
    
    if row["direct_match_exclusive"] or row["direct_match_non_exclusive"]:
        exclusive = row["direct_match_exclusive"]
        exclusive = exclusive if exclusive else []

        non_exclusive = row["direct_match_non_exclusive"]
        non_exclusive = non_exclusive if non_exclusive else []

        direct_match_overlapping.append(len(non_exclusive) - len(exclusive))

print("Mean IoU for matching reflection annotations:", round(statistics.mean(indirect_ious), 2))
print("Mean IoU for matching vehicle annotations:", round(statistics.mean(direct_ious), 2))
print()
print("Mean number of matching reflection annotations per image:", round(statistics.mean(num_matches_non_exclusive_indirect), 2))
print("Mean number of matching vehicle annotations per image:", round(statistics.mean(num_matches_non_exclusive_direct), 2))
print()
print("Mean number of unmatching reflection annotations per image:", round(statistics.mean(num_unmatches_non_exclusive_indirect), 2))
print("Mean number of unmatching vehicle annotations per image:", round(statistics.mean(num_unmatches_non_exclusive_direct), 2))


Mean IoU for matching reflection annotations: 0.41
Mean IoU for matching vehicle annotations: 0.6

Mean number of matching reflection annotations per image: 2.8
Mean number of matching vehicle annotations per image: 1.01

Mean number of unmatching reflection annotations per image: 2.11
Mean number of unmatching vehicle annotations per image: 0.19


In [9]:
print("Total number of reflection matches (non-exclusive):", sum(num_matches_non_exclusive_direct))
print("Total number of reflection UNmatches annotations (non-exclusive):", sum(num_unmatches_non_exclusive_direct))


Total number of reflection matches (non-exclusive): 1557
Total number of reflection UNmatches annotations (non-exclusive): 288


### Number of matches exclusive vs. non-exclusive

In [10]:

print("Mean number of OVERLAPPING reflections among annotators:", round(statistics.mean(indirect_match_overlapping), 2))

print("Mean number of OVERLAPPING vehicles among annotators:", round(statistics.mean(direct_match_overlapping), 2))

Mean number of OVERLAPPING reflections among annotators: 0.64
Mean number of OVERLAPPING vehicles among annotators: 0.02


## Types

In [11]:
from data_analysis.core.meta import ReflectionType

img_info_df["reflection_type"] = None
for i, row in img_info_df.iterrows():
    if row["indirect_match_non_exclusive"]:
        row_types = []
        for combi, box_iou in row["indirect_match_non_exclusive"]:
            types = [inst.type for inst in combi]
            unique_types = set(types)
            if len(unique_types) != 1:
                t = "ambiguous"
            else:
                t = types[0]
            row_types.append(t)
        img_info_df.at[i, "reflection_type"] = row_types


#### Reflection types for matches

In [13]:

num_reflection_types = {k: 0 for k in ReflectionType}
num_reflection_types["ambiguous"] = 0

for i, row in img_info_df.iterrows():
    if row["reflection_type"]:
        for t in row["reflection_type"]:
            num_reflection_types[t] += 1

total = 0
for k, v in num_reflection_types.items():
    total += v

print()
print("Total:", total)

for k, v in num_reflection_types.items():
    if v > 0:
        print(k, round(100 * v / total, 2))



Total: 9407
ReflectionType.FLOOR 17.3
ReflectionType.CAR 65.04
ReflectionType.CURB 4.07
ReflectionType.OTHER 5.35
ambiguous 8.25


#### Reflection types for **all** annotations

In [14]:
num_reflection_types = {k: 0 for k in ReflectionType}
total = 0

for img_info in dataset.img_infos:
    for annot in img_info.annots.values():
        for reflection in annot.reflections:
            num_reflection_types[reflection.type] += 1
            total += 1

print("Total:", total)
for k, v in num_reflection_types.items():
    if v > 0:
        print(k, round(100 * v / total, 2))

Total: 25109
ReflectionType.FLOOR 16.12
ReflectionType.CAR 69.96
ReflectionType.CURB 5.08
ReflectionType.OTHER 8.84
